# This is a sandbox for setting up a run of the simulation on `PRIVATE_AUTO` only. 

* It can be run on a small subsample pandas DataFrame
* The dask implementation might work with `map_partition()` but also might not. 


In [5]:
import pandas as pd
import dask.dataframe as dd
import numpy as np
import replicaEVSE.load_curve as sim
import os
from dask.diagnostics import ProgressBar
import dask

%reload_ext autoreload
%autoreload 2



pd.set_option('display.max_columns', None)

datadir = '../../data'
mode = 'PRIVATE_AUTO'
simulation_id = 'dask'
test = False

#Created in the EIA_data_download.ipynb notebook
existing_load=pd.read_csv('../../data/EIA_demand_summary.csv') 

# read in the joined trips and population data sets
merged_ddf = dd.read_parquet(os.path.join(datadir, 'wa_pop_and_trips_sorted.parquet'))

# right now, only look at private auto trips
ddf = merged_ddf.loc[merged_ddf['mode'] == mode] # len = 27.12446 million



if test:
    ddf = dd.read_parquet(os.path.join(datadir, 'wa_pop_and_trips_subsample.parquet'))
    ddf = ddf.loc[ddf['mode'] == mode]
    simulation_id = 'test'


In [33]:
# client.close()

In [2]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=48)  # Launches a scheduler and workers locally
client = Client(cluster)  # Connect to distributed cluster and override default

client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 48
Total threads: 96,Total memory: 251.88 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43873,Workers: 48
Dashboard: http://127.0.0.1:8787/status,Total threads: 96
Started: Just now,Total memory: 251.88 GiB
Comm: tcp://127.0.0.1:35823,Total threads: 2
Dashboard: http://127.0.0.1:42623/status,Memory: 5.25 GiB
Nanny: tcp://127.0.0.1:43385,


### There is a dask implementation by wrapping the `simulate_person_load()` with a `@dask.delayed` decorator. 

This saves us a few lines of code. 

In [6]:
out = sim.simulate_person_load_dask(ddf, existing_load, 'base', managed=False)
# out.compute()

In [7]:
out.compute()

2023-05-04 20:38:53,659 - distributed.nanny.memory - WARNING - Worker tcp://127.0.0.1:34237 (pid=85989) exceeded 95% memory budget. Restarting...
2023-05-04 20:38:54,847 - distributed.nanny - WARNING - Restarting worker
2023-05-04 20:41:23,696 - distributed.nanny.memory - WARNING - Worker tcp://127.0.0.1:46641 (pid=85902) exceeded 95% memory budget. Restarting...
2023-05-04 20:41:25,018 - distributed.nanny - WARNING - Restarting worker
2023-05-04 20:43:48,610 - distributed.nanny.memory - WARNING - Worker tcp://127.0.0.1:43587 (pid=85981) exceeded 95% memory budget. Restarting...
2023-05-04 20:43:49,864 - distributed.nanny - WARNING - Restarting worker
2023-05-04 20:46:13,746 - distributed.nanny.memory - WARNING - Worker tcp://127.0.0.1:36625 (pid=85892) exceeded 95% memory budget. Restarting...
2023-05-04 20:46:14,644 - distributed.nanny - WARNING - Restarting worker


KilledWorker: Attempted to run task finalize-5ec8b205-cc5c-4a58-b128-50441c289589 on 3 different workers, but all those workers died while running it. The last worker that attempt to run the task was tcp://127.0.0.1:36625. Inspecting worker logs is often a good next step to diagnose what went wrong. For more information see https://distributed.dask.org/en/stable/killed.html.

In [19]:
out.visualize()

RuntimeError: No visualization engine detected, please install graphviz or ipycytoscape

### Restack the DataFrames

In [ ]:
# restack the dataframes
charges_list = [x['charges'] for x in out]
loads_list = [x['loads'] for x in out]

Pandas significantly faster since we can fit these in memory. 

In [ ]:
# restack the dataframes
charges_df = pd.concat(charges_list)
loads_df = pd.concat(loads_list) # huge ~200 million rows

In [ ]:
loads_df

,person_id,load_segment_id,charge_id,window_start_time,window_end_time,load_kW
0,1000016319449981116,7318818662359676746_base_0,7318818662359676746_base,0 days 00:00:00,0 days 01:00:00,7.200000
1,1000016319449981116,7318818662359676746_base_1,7318818662359676746_base,0 days 01:00:00,0 days 02:00:00,7.200000
2,1000016319449981116,7318818662359676746_base_2,7318818662359676746_base,0 days 02:00:00,0 days 03:00:00,1.647283
3,1000016319449981116,7318818662359676746_base_3,7318818662359676746_base,0 days 03:00:00,0 days 04:00:00,0.000000
4,1000016319449981116,7318818662359676746_base_4,7318818662359676746_base,0 days 04:00:00,0 days 05:00:00,0.000000
...,...,...,...,...,...,...
19,9999319521751741792,1701538494003822315_base_19,1701538494003822315_base,0 days 19:00:00,0 days 20:00:00,0.000000
20,9999319521751741792,1701538494003822315_base_20,1701538494003822315_base,0 days 20:00:00,0 days 21:00:00,0.000000
21,9999319521751741792,1701538494003822315_base_21,1701538494003822315_base,0 days 21:00:00,0 days 22:00:00,0.000000
22,9999319521751741792,1701538494003822315_base_22,1701538494003822315_base,0 days 22:00:00,0 days 23:00:00,0.000000


In [ ]:
charges_df.to_parquet(os.path.join(datadir, f'charges_{mode}_{simulation_id}.parquet'))
loads_df.to_parquet(os.path.join(datadir, f'loads_{mode}_{simulation_id}.parquet'))

NameError: name 'charges_df' is not defined